In [47]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import RidgeClassifier,Ridge
from sklearn.model_selection import cross_val_score
pd.options.display.max_columns = 200
plt.rcParams['figure.figsize'] = (20, 12)

In [48]:
train = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\train.csv")
test = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\test.csv")

In [49]:
# sns.heatmap(train.corr(), annot = True, annot_kws = {"size":11}, linewidths = 1)

In [50]:
# 이상치 제거 1 - 분포

train = train.drop(train[train["u"] < -10].index) 
train = train.drop(train[(train["g"] < -10)].index)
train = train.drop(train[train["r"] > 60].index)
train = train.drop(train[train["i"] < -10].index)
train = train.drop(train[train["z"] < -6000].index)
train = train.drop(train[train["dered_i"] > 30].index)
train = train.drop(train[train["dered_z"] > 30].index)
train = train.drop(train[train["dered_z"] < 12].index)

# plt.figure(figsize=(20,12))
# sns.boxplot(train["class"], train["dered_z"])
# sns.countplot(alldata["nObserve"], alldata["class"])


# # 이상치 제거 2 - 상관도
train = train.drop(train[(train["g"] > 80) | (train["g"] < 5)].index)
train = train.drop(train[train["dered_u"] < 5].index)
train = train.drop(train[train["redshift"] < -100].index)
train = train.drop(train[train["z"] > 6000].index)
train = train.drop(train[train["r"] > 38].index)
train = train.drop(train[(train["redshift"] < -40) & (train["nObserve"] > 5)].index)

# plt.figure(figsize=(20,12))
# sns.scatterplot(train["r"], train["dered_i"])

In [51]:
alldata = pd.concat([train,test])
alldata

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
0,0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0.0
1,1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1.0
2,2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0.0
3,3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0.0
4,4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,399995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.185843,1.186322,1.185493,1.185652,1.186067,NaN
79996,399996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,1.284956,1.290772,1.279329,1.282120,1.287838,NaN
79997,399997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.188475,1.188563,1.188518,1.188480,1.188503,NaN
79998,399998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.180429,1.181329,1.179654,1.180025,1.180862,NaN


In [52]:
alldata["u-g"] = alldata["u"]-alldata["g"]
alldata["g-r"] = alldata["g"]-alldata["r"]
alldata["r-i"] = alldata["r"]-alldata["i"]
alldata["i-z"] = alldata["i"]-alldata["z"]

alldata["d1"] = alldata["dered_u"]-alldata["dered_g"]
alldata["d2"] = alldata["dered_g"]-alldata["dered_r"]
alldata["d3"] = alldata["dered_r"]-alldata["dered_i"]
alldata["d4"] = alldata["dered_i"]-alldata["dered_z"]

# alldata["minusOD"] = alldata["nObserve"] - alldata["nDetect"]
# alldata["minusOD"] = np.log1p(alldata["minusOD"])
# alldata["nObserve"] = np.log1p(alldata["nObserve"])
# alldata["nDetect"] = np.log1p(alldata["nDetect"])

alldata

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u-g,g-r,r-i,i-z,d1,d2,d3,d4
0,0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0.0,2.927183,1.327273,1.337061,0.732833,2.86647,1.30267,1.32301,0.72317
1,1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1.0,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157
2,2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0.0,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565
3,3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0.0,4.471910,0.967569,0.326209,0.260300,4.35900,0.90738,0.29262,0.23947
4,4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0.0,3.754259,1.656803,0.718216,0.341503,3.33756,1.57085,0.67271,0.31436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,399995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.185843,1.186322,1.185493,1.185652,1.186067,NaN,1.011727,0.390873,0.148287,0.059829,0.94283,0.30731,0.09990,0.01911
79996,399996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,1.284956,1.290772,1.279329,1.282120,1.287838,NaN,0.794588,0.119586,-0.396179,0.159076,0.75943,0.09486,-0.40563,0.16801
79997,399997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.188475,1.188563,1.188518,1.188480,1.188503,NaN,1.157989,0.080584,-0.037262,-0.004078,1.03826,-0.02936,-0.09957,-0.03574
79998,399998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.180429,1.181329,1.179654,1.180025,1.180862,NaN,1.298879,0.467991,0.310531,0.227785,1.28355,0.43231,0.28972,0.20745


In [53]:
alldata[alldata["z"]<0]

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u-g,g-r,r-i,i-z,d1,d2,d3,d4
56387,56387,17.439817,27.127991,18.039221,16.287418,-9.254774,-0.000129,17.54345,29.46140,18.99746,16.30331,24.63926,4,4,1.315968,1.325579,1.306602,1.311251,1.320737,0.0,-9.688174,9.088770,1.751804,25.542191,-11.91795,10.46394,2.69415,-8.33595
249594,249594,14.903783,18.158952,15.329157,14.472461,-8.099810,0.199247,14.72882,18.12766,15.23763,14.40336,22.80396,2,2,1.230272,1.222647,1.238085,1.234150,1.226434,2.0,-3.255169,2.829795,0.856696,22.572271,-3.39884,2.89003,0.83427,-8.40060
268213,268213,15.973551,15.748830,15.625340,17.243977,-5.728722,0.001004,15.93490,15.70637,15.60149,17.29350,22.88469,1,1,1.101586,1.102573,1.100663,1.101117,1.102071,1.0,0.224721,0.123490,-1.618637,22.972700,0.22853,0.10488,-1.69201,-5.59119
47170,367170,14.990991,-51.175280,12.035324,11.584408,-39.527151,-0.000162,21.04551,24.94197,11.91907,11.49891,22.76068,15,4,1.211727,1.213087,1.210504,1.211098,1.212389,NaN,66.166271,-63.210604,0.450916,51.111559,-3.89646,13.02290,0.42016,-11.26177
74455,394455,17.737915,-2.106023,-5.438650,19.225011,-0.904629,0.016621,17.66523,24.68772,24.75778,20.29871,22.49606,1,1,1.193723,1.194992,1.192587,1.193139,1.194341,NaN,19.843937,3.332628,-24.663661,20.129640,-7.02249,-0.07006,4.45907,-2.19735


In [54]:
alldata["g"] = alldata["g"]+52
alldata["z"] = alldata["z"]+39.6
alldata["i"] = alldata["i"]+5.5

In [55]:
alldata[["g","i","z"]] = alldata[["g","i","z"]].apply(np.log1p)
alldata

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u-g,g-r,r-i,i-z,d1,d2,d3,d4
0,0,23.263956,4.295062,19.009500,3.185213,4.052474,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0.0,2.927183,1.327273,1.337061,0.732833,2.86647,1.30267,1.32301,0.72317
1,1,15.052147,4.205617,13.452418,2.984087,3.984696,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1.0,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157
2,2,16.786385,4.231573,15.536318,3.086191,4.024459,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0.0,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565
3,3,25.660638,4.306612,20.221158,3.273173,4.098248,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0.0,4.471910,0.967569,0.326209,0.260300,4.35900,0.90738,0.29262,0.23947
4,4,24.453429,4.299992,19.042368,3.211817,4.070439,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0.0,3.754259,1.656803,0.718216,0.341503,3.33756,1.57085,0.67271,0.31436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,399995,19.673388,4.271956,18.270788,3.203661,4.071804,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.185843,1.186322,1.185493,1.185652,1.186067,NaN,1.011727,0.390873,0.148287,0.059829,0.94283,0.30731,0.09990,0.01911
79996,399996,21.522869,4.300386,20.608695,3.314363,4.118155,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,1.284956,1.290772,1.279329,1.282120,1.287838,NaN,0.794588,0.119586,-0.396179,0.159076,0.75943,0.09486,-0.40563,0.16801
79997,399997,19.399065,4.266070,18.160492,3.206712,4.074173,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.188475,1.188563,1.188518,1.188480,1.188503,NaN,1.157989,0.080584,-0.037262,-0.004078,1.03826,-0.02936,-0.09957,-0.03574
79998,399998,18.899747,4.257042,17.132878,3.149412,4.046459,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.180429,1.181329,1.179654,1.180025,1.180862,NaN,1.298879,0.467991,0.310531,0.227785,1.28355,0.43231,0.28972,0.20745


In [56]:
alldata2 = alldata.drop(["id","class","airmass_u","airmass_g","airmass_r","airmass_i","airmass_z"], axis=1)
alldata2

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,u-g,g-r,r-i,i-z,d1,d2,d3,d4
0,23.263956,4.295062,19.009500,3.185213,4.052474,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,2.927183,1.327273,1.337061,0.732833,2.86647,1.30267,1.32301,0.72317
1,15.052147,4.205617,13.452418,2.984087,3.984696,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157
2,16.786385,4.231573,15.536318,3.086191,4.024459,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565
3,25.660638,4.306612,20.221158,3.273173,4.098248,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,4.471910,0.967569,0.326209,0.260300,4.35900,0.90738,0.29262,0.23947
4,24.453429,4.299992,19.042368,3.211817,4.070439,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,3.754259,1.656803,0.718216,0.341503,3.33756,1.57085,0.67271,0.31436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,19.673388,4.271956,18.270788,3.203661,4.071804,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.011727,0.390873,0.148287,0.059829,0.94283,0.30731,0.09990,0.01911
79996,21.522869,4.300386,20.608695,3.314363,4.118155,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,0.794588,0.119586,-0.396179,0.159076,0.75943,0.09486,-0.40563,0.16801
79997,19.399065,4.266070,18.160492,3.206712,4.074173,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.157989,0.080584,-0.037262,-0.004078,1.03826,-0.02936,-0.09957,-0.03574
79998,18.899747,4.257042,17.132878,3.149412,4.046459,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.298879,0.467991,0.310531,0.227785,1.28355,0.43231,0.28972,0.20745


In [57]:
rs = RobustScaler()
alldata2 = rs.fit_transform(alldata2)

# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# alldata2 = rs.fit_transform(alldata2)



In [58]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

# 로지스틱

In [59]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(n_jobs = -1)
acc = cross_val_score(lr, train2, train["class"], cv=10, n_jobs = -1, scoring = "accuracy").mean()
print(acc)  

lr.fit(train2,train["class"])
logistic_proba = lr.predict_proba(test2)

# 
# 로지스틱회귀
# 0.83681  이상치 제거 전
# 0.83686  이상치 제거 후

# 0.8474  로그 씌운 후




0.8474061735593745


In [60]:
logistic_model = pd.DataFrame(logistic_proba)
logistic_model.to_csv("logistic_proba.csv", index= False)

# 릿지

In [63]:
for i in [0.01,0.1, 1,5,10]:
    ridgecv = RidgeClassifier(alpha = [i]) 
    acc = cross_val_score(ridgecv, train2, train["class"], cv=10, n_jobs = -1, scoring = "accuracy").mean()
    print(acc)  
 
ridge = RidgeClassifier(alpha = [0.01])
ridge.fit(train2,train["class"])
ridge_proba = ridge.predict(test2)

# 0.66013  이상치 제거 전
# 0.66696   이상치 제거 후
# 0.6734  로그


0.6734701607479161
0.6734295151319837
0.6731324876121894
0.6719756431658283
0.6703842013399532


In [64]:
ridge_model = pd.DataFrame(ridge_proba)
ridge_model.columns = ["ridge_result"]
ridge_model.to_csv("ridge_result.csv", index= False)

In [ ]:
# sub = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\sample_submission.csv")
# sub

In [ ]:
# sub["class"] = result
# sub

In [ ]:
# sub.to_csv("dk_dacon_linear.csv", index=False)